In [ ]:
import random
import time
import cv2
from keras.models import load_model
import numpy as np

model = load_model('keras_model.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)



def game_options(computer,user):
    if user == 'Rock' and computer == 'Paper':
        message = 'You chose rock, computer chose paper, You Lost!'
    elif user == 'Rock' and computer == 'Scissors':
        message = 'You chose rock, computer chose scissors, You Won!'
    elif user == 'None':
        message = 'You Chose Nothing, You Lost!'
    elif user == 'Paper' and computer == 'Rock':
        message = 'You chose paper, computer chose rock, You Won!'
    elif user == 'Paper' and computer == 'Scissors':
        message = 'You chose paper, computer chose scissors, You Lost!'
    elif user == 'Scissors' and computer == 'Rock':
        message = 'You chose scissors, computer chose rock, You Lost!'
    elif user == 'Scissors!' and computer == 'Paper':
        message = 'You chose scissors, computer chose paper, You Won!'
    else:
        message = "It's a Draw!"
    return message

def get_user_choice(prediction):
    if(prediction[0][0]) > 0.5:
        user = ('Rock')
    elif(prediction[0][1]) > 0.5:
        user = ('Paper')
    elif(prediction[0][2]) > 0.5:
        user = ('Scissors')
    else:
        user = ('None')
    return user

started = False
countdown = False
show_message = False
elapsed = 0
counter = 0

user_wins = 0
computer_wins = 0
message = ''

while True: 
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)

    if prediction[0][0] > 0.5:
        print("Rock")
    elif prediction[0][1] > 0.5:
        print("Paper")
    elif prediction[0][2] > 0.5:
        print("Scissors")
    else:
        print("None")


    if not started:
        message = 'Press b to Begin'
        if cv2.waitKey(1)  == ord('b'):
            started = True
            countdown = True
            counter = time.time()
            

    if started:
        elapsed = 5 - (time.time() - counter) # 5 second countdown
        
        if countdown:
            message = f"Choose Rock, Paper or Scissors in {int(elapsed)} seconds"

        if elapsed <= -4: # when countdown reaches -4 secs, win tally is displayed, game ends at 3 wins.
            if computer_wins == 3:
                message = f"Your Wins: {user_wins} Computer Wins: {computer_wins} Game Over You Lose!"
            elif user_wins == 3:
                message = f"Your Wins: {user_wins} Computer Wins: {computer_wins} You Won!"
            else:
                message = f"Your Wins: {user_wins} Computer Wins: {computer_wins} Press n to continue"
            if cv2.waitKey(1) == ord('n'): # hit 'n' key to initiate new round.
                started = False
                elapsed = 0
                show_message = False

        elif elapsed <= 0: # when countdown reaches 0 secs, predictions are compared, winner gains a point.
            countdown = False
            if show_message == False:
                user = get_user_choice(prediction)
                computer = random.choice(['Rock','Paper','Scissors'])
                message = game_options(computer, user)
                if 'You Won' in message:
                    user_wins += 1
                elif 'You Lost' in message:
                    computer_wins += 1
                show_message = True


    cv2.putText(frame, message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

